Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchain);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height], "weight" => df2[:weight_c]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD       Naive SE       MCSE      ESS
alpha 154.5931105 0.273419378 0.004323140 0.00446405279 1000
 beta   0.9045745 0.042280223 0.000668509 0.00065641163 1000
sigma   5.1001020 0.196573182 0.003108095 0.00254160492 1000

Quantiles:
          2.5%        25.0%      50.0%      75.0%        97.5%   
alpha 154.05600000 154.420000 154.586000 154.7682500 155.14205000
 beta   0.82324235   0.877102   0.904326   0.9325775   0.99047902
sigma   4.73940825   4.962275   5.096675   5.2302025   5.50404225



Save the chains in a JLS file

In [8]:
serialize("m4.4s.jls", chn)

chn2 = deserialize("m4.4s.jls")

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Union{Missing, Float64}[154.566 1.0142 5.07915; 154.88 0.923332 5.35023; … ; 154.536 0.911454 5.23292; 154.776 0.939177 5.09607]

Union{Missing, Float64}[154.682 0.900258 5.04388; 154.713 0.829459 4.99047; … ; 154.683 0.940769 4.74777; 154.933 0.930478 5.18775]

Union{Missing, Float64}[154.776 0.920041 4.80317; 154.877 0.890345 5.06913; … ; 154.451 0.833377 5.12158; 154.407 0.851211 5.06065]

Union{Missing, Float64}[154.762 0.916721 5.02543; 154.213 0.868161 5.026; … ; 154.535 0.854563 5.12529; 154.69 0.913839 5.32278]

Should be identical to earlier result

In [9]:
describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD       Naive SE       MCSE      ESS
alpha 154.5931105 0.273419378 0.004323140 0.00446405279 1000
 beta   0.9045745 0.042280223 0.000668509 0.00065641163 1000
sigma   5.1001020 0.196573182 0.003108095 0.00254160492 1000

Quantiles:
          2.5%        25.0%      50.0%      75.0%        97.5%   
alpha 154.05600000 154.420000 154.586000 154.7682500 155.14205000
 beta   0.82324235   0.877102   0.904326   0.9325775   0.99047902
sigma   4.73940825   4.962275   5.096675   5.2302025   5.50404225



End of `m4.4s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*